# An introduction to non-revocation proofs using cryptographic accumulators

The purpose of this notebook is to give a high level idea of how revocation works using cryptographic accumulators. In reality, there are various ways to construct and use accumulators and to generate ZKP of non-revocation. Below is just meant to provide a very simple example on how the idea works.

## Basic workings of a non-revocation proof

In [1]:
# !pip install ECPy
# !pip install pycrypto

In [2]:
from ecpy.curves import Curve, Point
from Crypto.Util import number
import numpy as np

In [3]:
c = Curve.get_curve('secp256k1')
G = c.generator
order = c.order

The issuer generates a tails file and computes the accumulator

In [4]:
tails_factors = []
tails_factors_inv = []
for i in range(10):
    tmp = number.getPrime(255) #one less than order to make sure that it does not wrap around the mod
    tails_factors.append(tmp)
    tails_factors_inv.append(tmp)
    
H = tails_factors[0] * G # Holder's point
w = np.prod(tails_factors[1:]) % order # Witness value
A = H * w # Accumulator

We issue a credential where the non revocation proof includes a value from the index. For our example, we use index 0. The witness includes all the factors except the holder's point. The accumulator is published.

A simple demonstration of non-revocation can be done as follows:

* Generator is $G$
* Holder point is $H = a_0 \cdot G$
* Tails file is $[a_0, a_1, \ldots, a_n]$
* Accumulator is $A = G \cdot \prod_{i=0}^{n}a_i$
* Witness point for $a_0$ is $w = a_0^{-1} \cdot \prod_{i=0}^{n}a_i$
* The holder generates a random blinding factor $r$ and its inverse $r^{-1}$.

The holder now has everything required to initiate the non revocation proof.

* The holder sends to the verifier: $[r \cdot H, r \cdot w]$. The verifier cannot learn $(H,w)$ as that would require an efficient way to solve the EC discrete log problem.
* The verifier generates a challenge $c$ and sends back $[c \cdot r \cdot H, c \cdot r \cdot w]$. Note that the holder cannot learn $c$ due to ECDLP.
* The holder computes $[c \cdot r \cdot H \cdot r^{-1}, c \cdot r \cdot w \cdot r^{-1}]$, which gives the non-revocation proof: $p = (c \cdot r \cdot H \cdot r^{-1} ) (c \cdot r \cdot w \cdot r^{-1}) = c^2 \cdot w \cdot H$ 
* Victor checks if $p \cdot c^{-2} = A$ 

In [5]:
# Holder
r = number.getRandomRange(2, order)
r_inv = pow(r, -1, order)
blinded_values = [r * H, r * w]

# Verifier does
c = number.getRandomRange(2, order)
challenge = [c * i for i in blinded_values]

# Holder does
non_revocation = (r_inv * challenge[0]) * (r_inv * challenge[1])

# Verifier now checks
non_revocation * pow(c, -2, order) == A

True

## Issuer revocation

### Issuer revokes a credential 

* To remove $H$ from $A$, the issuer simply does $A_{new} = A_{old} \cdot a_0^{-1}$.
* To remove any value set from $A$, the issuer computes $A_{new} = A_{old} \cdot \prod^{n}_{i=m} a_i^{-1}$ where the range $[m,n]$ is the index of all the revoked values.
* When a value is revoked, the issuer publishes a delta to allow still valid holders to prove non-revocation.

Note that it is trivially easy for a holder create a false proof. This because the delta contains information about what is still a valid point. As stated, the above example is to create a basic understanding of how accumulators work. The example below is closer to the truth.

In [6]:
delta = np.prod(tails_factors_inv[5:]) % order
A_new = A * delta

In [7]:
# Holder
r = number.getRandomRange(2, order)
r_inv = pow(r, -1, order)
blinded_values = [r * H, r * w]

# Verifier does
c = number.getRandomRange(2, order)
challenge = [c * i for i in blinded_values]

# Holder does
non_revocation = (r_inv * challenge[0]) * (r_inv * challenge[1]) * delta

# Verifier now checks
non_revocation * pow(c, -2, order) == A_new

True